In [1]:
import numpy as np
import sympy as sym
import numba
import pydae.build as db

\begin{equation}
\begin{split}
\sf  \frac{d\delta}{dt} &= \sf \Omega_b \left(\omega - \omega_s\right)\\
\sf  \frac{d\omega}{dt} &= \sf \frac{1}{2H} \left(p_m - p_e - D (\omega - \omega_s) \right)\\
\end{split}
\end{equation}


\begin{equation}
\begin{split}
    \sf g_1 &= \sf - p_e + \left( v_q + R_a i_q  \right) i_q + \left( v_d + R_a i_d  \right) i_d = 0\\
    \sf g_2 &= \sf  v_q + R_a i_q  - e'_q + \left( X'_d - X_l  \right) i_d  = 0\\
    \sf g_3 &= \sf  v_d + R_a i_d  -  \left( X'_d - X_l  \right) i_q = 0\\
    \sf g_4 &= \sf  v_d - V \sin  \left( \delta - \theta \right)  \\
	\sf g_5 &= \sf  v_q - V \cos  \left( \delta - \theta \right)  \\
	\sf g_6 &= \sf  v_d i_d + v_q i_q -P \\
	\sf g_7 &= \sf  v_q i_d - v_d i_q -Q \\
	\sf g_8 &= \sf  P - (v_1*v_0*sin(theta_1 - theta_0))/X_g \\
	\sf g_9 &= \sf  Q + (v_1*v_0*cos(theta_1 - theta_0))/X_g - v_1^2/X_g
\end{split}
\end{equation}

	

\begin{equation}
\begin{split}
    \sf  \mathbf{x} &= \sf \left[\delta \;\;\; \omega \right]^T
\end{split}
\end{equation}

\begin{equation}
\begin{split}
     \sf \mathbf{y} &= \sf \left[p_e\;\;i_d\;\;i_q\;\;v_d\;\;v_q\;\;P\;\;Q\;\;v_1\;\;\theta_1\right]^T
\end{split}
\end{equation}

\begin{equation}
\begin{split}
    \sf  \mathbf{f} &= \sf \left[\frac{d\delta}{dt} \;\;\; \frac{d\omega}{dt} \right]^T
\end{split}
\end{equation}

\begin{equation}
\begin{split}
     \sf \mathbf{g} &= \sf \left[g_1\;\;g_2\;\;g_3\;\;g_4\;\;g_5\;\;g_6\;\;g_7\;\;g_8\;\;g_9\right]^T
\end{split}
\end{equation}

## System definition 

In [13]:
params = {'X_d':1.81,'X1d':0.3,'R_a':0.003, 
          'H':3.5,'D':1.0,
          'Omega_b':2*np.pi*50,'omega_s':1.0,
          'X_g':0.05,'v_0':1.0,'theta_0':0.0}


u_ini_dict = {'P':0.8, 'Q':0.2}  # for the initialization problem
u_run_dict = {'p_m':0.8,'v_f':1.0}  # for the running problem (here initialization and running problem are the same)


x_list = ['delta','omega']    # [inductor current, PI integrator]
y_ini_list = ['p_e','i_d','i_q','v_d','v_q','p_m','e1q','v_1','theta_1'] # for the initialization problem
y_run_list = ['p_e','i_d','i_q','v_d','v_q','P','Q','v_1','theta_1'] # for the running problem (here initialization and running problem are the same)



sys_vars = {'params':params,
            'u_list':u_run_dict,
            'x_list':x_list,
            'y_list':y_run_list}

exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

In [14]:
ddelta = Omega_b*(omega - omega_s)
domega = 1/(2*H)*(p_m - p_e - D*(omega - omega_s))


g_1 =  -p_e + i_d*(v_d + R_a*i_d) + i_q*(v_q + R_a*i_q)  
g_2 = v_q + R_a*i_q + X1d*i_d - e1q
g_3 = v_d + R_a*i_d - X1d*i_q  
g_4 = -v_d +  v_1*sin(delta - theta_1) 
g_5 = -v_q + v_1*cos(delta - theta_1)                                             
g_6 = i_d*v_d + i_q*v_q - P
g_7 = i_d*v_q - i_q*v_d - Q
g_8 = P - (v_1*v_0*sin(theta_1 - theta_0))/X_g
g_9 = Q + (v_1*v_0*cos(theta_1 - theta_0))/X_g - v_1**2/X_g
                                            
h_1 = p_m

sys = {'name':'smib_milano_ex8p1_2ord',
       'params':params,
       'f':[ddelta,domega],
       'g':[g_1,g_2,g_3,g_4,g_5,g_6,g_7,g_8,g_9],
       'g_ini':[g_1,g_2,g_3,g_4,g_5,g_6,g_7,g_8,g_9],
       'x':x_list,
       'y_ini':y_ini_list,
       'y':y_run_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h':[h_1]}

sys = db.system(sys)
db.sys2num(sys)
                